In [51]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the URL
url = 'https://satr.codes/list/courses'
driver.get(url)

# Wait for the page to load
time.sleep(5)

# Function to scroll to the bottom of the page
def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Function to click the "Load More" button if it exists
def click_load_more(times=6):
    for _ in range(times):
        try:
            scroll_to_bottom()
            time.sleep(2)
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'load-more-button'))
            )
            load_more_button.click()
            time.sleep(3)  # Wait for new content to load
        except Exception as e:
            print("No more 'Load More' buttons or an error occurred:", e)
            break

# Ensure all courses are loaded initially
click_load_more()

# Function to click an element with retries
def click_element(element, retries=3):
    for attempt in range(retries):
        try:
            driver.execute_script("arguments[0].scrollIntoView();", element)
            time.sleep(1)
            element.click()
            return True
        except Exception as e:
            print(f"Retry {attempt + 1}/{retries} failed: {e}")
            time.sleep(2)
    return False

# Function to extract courses
def extract_courses():
    body_content = driver.find_element(By.TAG_NAME, 'body').get_attribute('innerHTML')
    soup = BeautifulSoup(body_content, 'html.parser')
    course_cards = soup.find_all("div", class_="course-card-detail-name")
    return [card.get_text(strip=True) for card in course_cards]

# Initial extraction of courses
course_names = extract_courses()

# Initialize the lists to store the extracted text
C_name = []
hours_levels = []
dec = []
C_info = []
learning_goals = []
span_contents = []

# Iterate through the course names and click each one
for index, course in enumerate(course_names):
    print(f"Processing course {index + 1}/{len(course_names)}: {course}")
    try:
        # Find elements by class name
        elements = driver.find_elements(By.CLASS_NAME, "course-card-detail-name")
        for element in elements:
            if element.text == course:
                if click_element(element):
                    time.sleep(5)  # Wait for the page to load
                    
                    # Extract course details after clicking
                    body_content = driver.find_element(By.TAG_NAME, 'body').get_attribute('innerHTML')
                    soup = BeautifulSoup(body_content, 'html.parser')

                    # Extract course name
                    course_name_div = soup.find_all("li", class_="breadcrumb-item pointer ng-star-inserted")[1]
                    if course_name_div:
                        C_name.append(course_name_div.text.strip())

                    # Extract course description
                    container_div = soup.find('div', id='cdk-describedby-message-container')

                    # Loop through all div elements inside the container
                    if container_div:
                        message_divs = container_div.find_all('div', role='tooltip')
                        for message_div in message_divs:
                            extracted_text = message_div.text.strip()
                            dec.append(extracted_text)

                    # Extract course information from the table
                    table = soup.find('table')
                    if table:
                        # Extract information from the table
                        info_elements = table.find_all('td', class_='course-information-icon-text')
                        
                        # Create a list of the extracted information
                        info_list = [td.text.strip() for td in info_elements]
                        
                        # Append the list of information to C_info
                        C_info.append(info_list)

                    # Extract learning goals from <li> elements
                    learning_goals_list = [
                        [p_element.text.strip() 
                         for li in soup.find_all('li', class_='ng-star-inserted') 
                         if (p_element := li.find('p', class_='text'))
                        ]
                    ]
                    
                    # Append the list of lists to learning_goals
                    learning_goals.append(learning_goals_list)

                    span_elements = [span.get_text(separator=" ", strip=True) for span in soup.find_all('span', class_='mat-content')]
                    span_contents.append(span_elements)

                    hour_divs = [div.text.strip() for div in soup.find_all('div', class_='course-card-bottom-text')]
                    hours_levels.append(hour_divs)
                    
                    driver.back()  # Navigate back to the previous page
                    time.sleep(2)  # Wait a bit for the page to load again

                    
                    click_load_more(times=6)
                    course_names = extract_courses()  # Update the course list

                else:
                    print(f"Failed to click element {course} after retries.")
                break  # Exit the inner loop and re-find elements
    except Exception as e:
        print(f"Error processing course {course}: {e}")

# Close the browser
driver.quit()

# Output the extracted data for verification
print("Course Names:", C_name)
print("Course Descriptions:", dec)
print("Course Information:", C_info)
print("Learning Goals:", learning_goals)
print("Span Contents:", span_contents)
print("Hours and Levels:", hours_levels)


Processing course 1/125: مقدمة في المشاريع التقنية
Processing course 2/125: تعلم الآلة (ML) - 101
Processing course 3/125: تحليل البيانات الاستكشافي (E...
Processing course 4/125: تمثيل البيانات: مكتبة Seaborn
Processing course 5/125: تمثيل البيانات: مكتبة Matplo...
Processing course 6/125: تحليل ومعالجة البيانات: مكتب...
Processing course 7/125: JAVA 104
Processing course 8/125: JAVA 103
Processing course 9/125: JAVA 102
Processing course 10/125: تحليل ومعالجة البيانات: مكتب...
Processing course 11/125: مقدمة في Object Oriented Pro...
Processing course 12/125: JAVA 101
Processing course 13/125: مقدمة في البرمجة Intro to Pr...
Processing course 14/125: Unity 101
Processing course 15/125: JSON 101
Processing course 16/125: C++ 104
Processing course 17/125: CI/CD 102
Processing course 18/125: C++ 103
Processing course 19/125: CI/CD 101
Processing course 20/125: C++ 102
No more 'Load More' buttons or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79E491F52+60322]
	(

In [42]:
len(hours_levels)

40

In [34]:
dec

['سنتعرف في هذة الدورة على ماهية المشاريع التقنية ، من ثم عناصرها الى ان ننتقل لدورة حياة المشاريع التقنية وانوعها وختاما سنتعرف على العمليات والبرامج والمحافظ في المجال التقني',
 'يهدف الدورة للتعرف على مفهوم تعلم الآلة (Machine Learning)، وأنوعه وتطبيقاته، كما سيتم التعرف على مراحل بناء نماذج تعلم الآلة وما هي خوارزميات تعلم الآلة.',
 'تهدف الدورة للتعرف على خطوات تحليل البيانات الاستكشافي (EDA - Exploratory Data Analysis) من عدة زوايا وعدة تحليلات والتي تساعد بدورها في فهم البيانات بشكل أكبر.',
 'تهدف الدورة للتعرف على ما هي مكتبة Seaborn، وما هي أنواع الرسوم البيانية وتمثيل وتعديل خصائص البيانات باستخدام الرسوم البيانية التابعة لها،',
 'تهدف الدورة للتعرف على كيفية تمثيل البيانات باستخدام الرسوم البيانية التابعة لمكتبة Matplotlib.',
 'في هذه الدورة سنتعرف على مكتبة Pandas التي تساعدنا على التعامل مع البيانات ومعالجتها وتحليلها',
 'تهدف الدورة  لتطبيق ما تم التعرف عليه في الدورات السابقة في مشروع برمجي بلغة JAVA.',
 'تهدف الدورة للتعرف على بعض المفاهيم التي تتعلق بالبرمجة كائنية الت

In [25]:
description_div

In [35]:
C_name

['مقدمة في المشاريع التقنية',
 'تعلم الآلة (ML) - 101',
 'تحليل البيانات الاستكشافي (EDA)',
 'تمثيل البيانات: مكتبة Seaborn',
 'تمثيل البيانات: مكتبة Matplotlib',
 'تحليل ومعالجة البيانات: مكتبة Pandas',
 'JAVA 104',
 'JAVA 103',
 'JAVA 102',
 'تحليل ومعالجة البيانات: مكتبة Pandas',
 'مقدمة في Object Oriented Programming (OOP)',
 'JAVA 101',
 'مقدمة في البرمجة Intro to Programming',
 'Unity 101',
 'JSON 101',
 'C++ 104',
 'CI/CD 102',
 'C++ 103',
 'CI/CD 101',
 'C++ 102',
 'PHP OOP 104',
 'Dart 103',
 'C++ 101',
 'Dart 102',
 'PHP Array 103',
 'Unit Test 102',
 'PHP Functions 102',
 'PHP Basics 101',
 'Unit Test 101',
 'Code Documentation 102',
 'Code Documentation 101',
 'Secure Shell (SSH) 102']

In [36]:
C_info

[['120 دقيقة تدريبية', '5  اختبارات'],
 ['120 دقيقة تدريبية', 'مشروعان', '3  اختبارات'],
 ['120 دقيقة تدريبية', 'مشروع واحد', '4  اختبارات'],
 ['120 دقيقة تدريبية', 'مشروعان', 'اختباران'],
 ['120 دقيقة تدريبية', 'مشروعان', '7  اختبارات'],
 ['120 دقيقة تدريبية', 'مشروعان', '4  اختبارات'],
 ['180 دقيقة تدريبية'],
 ['120 دقيقة تدريبية', '11  مشروع', '5  اختبارات'],
 ['120 دقيقة تدريبية', '9  مشاريع', '4  اختبارات'],
 ['120 دقيقة تدريبية', 'مشروعان', '4  اختبارات'],
 ['120 دقيقة تدريبية', '5  مشاريع', '7  اختبارات'],
 ['120 دقيقة تدريبية', '7  مشاريع', '5  اختبارات'],
 ['180 دقيقة تدريبية', '9  مشاريع', '10  اختبار'],
 ['120 دقيقة تدريبية', 'مشروع واحد', 'اختباران'],
 ['120 دقيقة تدريبية', 'مشروع واحد', '4  اختبارات'],
 ['180 دقيقة تدريبية', '3  اختبارات'],
 ['60 دقيقة تدريبية', 'مشروع واحد', '3  اختبارات'],
 ['120 دقيقة تدريبية', '3  اختبارات'],
 ['120 دقيقة تدريبية', 'مشروع واحد', '3  اختبارات'],
 ['120 دقيقة تدريبية', '3  اختبارات'],
 ['180 دقيقة تدريبية', '7  اختبارات'],
 ['120 دقيقة ت

In [37]:
learning_goals

[[['ماهو المشروع وصفاته وأمثلة للمشاريع التقنية',
   'ماهي عناصر المشروع وتطبيق عملي لفهمهم',
   'دورات حياة المشاريع التقنية وانواعها',
   'ماهي العمليات والبرامج والمحافظ في المجال التقني']],
 [['أنواع خوارزميات تعلم الآلة.',
   'مراحل بناء نماذج تعلم الآلة.',
   'خوارزميات الانحدار (Regression Algorithms).']],
 [['التحليل الوصفي (Descriptive Analysis)',
   'تحليل متغير واحد (Univariate Analysis)',
   'تحليل متغيرين (Bivariate Analysis)',
   'تحليل عدة متغيرات (Multivariate Analysis)']],
 [['التعرف على مكتبة Seaborn.',
   'التعرف على أنواع الرسوم البيانية.',
   'التعرف على كيفية التعديل على خصائص الرسوم البيانية.']],
 [['التعرف على أنواع الرسوم البيانية.',
   'التعرف على كيفية التعديل على خصائص الرسوم البيانية.',
   'التعرف على كيفية إنشاء الرسوم البيانية على شكل Subplots.']],
 [['كيفية إنشاء إطار بيانات وتنفيذ العمليات عليه',
   'الوصول للبيانات واسترجاعها',
   'تطبيق أهم الدوال على البيانات']],
 [['تطبيق المفاهيم الأساسية للغة JAVA',
   'التعرف على الترتيب الصحيح لبناء مشروع بلغة J

In [52]:
data = {
    "C_name": C_name,
    "hours_levels":hours_levels,
    "dec": dec,
    "C_info": C_info,
    "learning_goals": learning_goals,
    "span_contents": span_contents
    
}

In [53]:
import pandas as pd
df = pd.DataFrame(data)

In [54]:
df

,C_name,hours_levels,dec,C_info,learning_goals,span_contents
0,مقدمة في المشاريع التقنية,"[مبتدئ, ساعتان]",سنتعرف في هذة الدورة على ماهية المشاريع التقني...,"[120 دقيقة تدريبية, 5 اختبارات]","[[ماهو المشروع وصفاته وأمثلة للمشاريع التقنية,...","[المحتوى التعليمي, منصاتنا, 1 . وصف الدورة, 2 ..."
1,تعلم الآلة (ML) - 101,"[متوسط, ساعتان]",يهدف الدورة للتعرف على مفهوم تعلم الآلة (Machi...,"[120 دقيقة تدريبية, مشروعان, 3 اختبارات]","[[أنواع خوارزميات تعلم الآلة., مراحل بناء نماذ...","[المحتوى التعليمي, منصاتنا, 1 . مقدمة عن تعلم ..."
2,تحليل البيانات الاستكشافي (EDA),"[متوسط, ساعتان]",تهدف الدورة للتعرف على خطوات تحليل البيانات ال...,"[120 دقيقة تدريبية, مشروع واحد, 4 اختبارات]","[[التحليل الوصفي (Descriptive Analysis), تحليل...","[المحتوى التعليمي, منصاتنا, 1 . مقدمة عن EDA, ..."
3,تمثيل البيانات: مكتبة Seaborn,"[مبتدئ, ساعتان]",تهدف الدورة للتعرف على ما هي مكتبة Seaborn، وم...,"[120 دقيقة تدريبية, مشروعان, اختباران]","[[التعرف على مكتبة Seaborn., التعرف على أنواع ...","[المحتوى التعليمي, منصاتنا, 1 . مقدمة عن مكتبة..."
4,تمثيل البيانات: مكتبة Matplotlib,"[متوسط, ساعتان]",تهدف الدورة للتعرف على كيفية تمثيل البيانات با...,"[120 دقيقة تدريبية, مشروعان, 7 اختبارات]","[[التعرف على أنواع الرسوم البيانية., التعرف عل...","[المحتوى التعليمي, منصاتنا, 1 . مقدمة عن مكتبة..."
...,...,...,...,...,...,...
115,JavaScript 102,"[متوسط, 6 ساعات]",سنتعرف في هذهِ الدورة على عدد من المفاهيم في J...,"[360 دقيقة تدريبية, مشروع واحد, 5 اختبارات]","[[طرق التعامل مع الأرقام ومفهوم Callback., أهم...","[المحتوى التعليمي, منصاتنا, 1 . الأرقام والتوا..."
116,JavaScript 103,"[متقدم, 6 ساعات]",سنتعرف في هذهِ الدورة على عدد من المفاهيم المت...,"[360 دقيقة تدريبية, 3 اختبارات]",[[البحث في النصوص باستخدام Regular Expression....,"[المحتوى التعليمي, منصاتنا, 1 . مفهوم Regular ..."
117,Node.js,"[مبتدئ, 3 ساعات]",سنتعرف في هذهِ الدورة على مفهوم Node.js بشـكل ...,"[180 دقيقة تدريبية, 6 اختبارات]","[[التعرف على مفهوم Node.js., التعرف على مفهوم ...","[المحتوى التعليمي, منصاتنا, 1 . مقدمة في Node...."
118,Swift 103,"[متوسط, 4 ساعات]",سنتعرف في هذهِ الدورة على مفاهيم متقدمة في Swi...,"[240 دقيقة تدريبية, مشروع واحد, 6 اختبارات]",[[مفهوم Enumeration و Tuple بالإضافة إلى Struc...,"[المحتوى التعليمي, منصاتنا, 1 . مقدمة في Enume..."


In [55]:
df.to_csv("Course.csv",index=False)